# Speech-to-Text with Whisper Transfer Learning

**Objective:** Fine-tune a Whisper base model on the United-Syn-Med dataset to improve medical speech transcription accuracy in a live teleconsultation context.

In [1]:
# Installing required packages

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer datasets torchaudio transformers accelerate soundfile

  Cloning https://github.com/openai/whisper.git to c:\users\ifedi\appdata\local\temp\pip-req-build-y7g0svdv
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 65.3/65.3 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 2.4 MB/s eta 0:00:00
     -------------------------------------- 893.9/893.9 kB 2.3 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     ---------------------------------------- 30.3/30.3 MB 2.0 MB/s eta 0:00:00
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl (274 kB)
  Created wheel for openai

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\ifedi\AppData\Local\Temp\pip-req-build-y7g0svdv'

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 491.5/491.5 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 10.5/10.5 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 362.1/362.1 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 1.3 MB/s eta 0:00:00
  Using cached click-8.2.1-py3-none-any.whl (102 kB)
     ---------------------------------------- 1.6/1.6 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 25.8/25.8 MB 1.9 MB/s eta 0:00:00
     -------------------------------------- 116.3/116.3 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 11.1/11.1 MB 2.6 MB/s eta 0:00:00
     -------------------------------------- 143.5/143.5 kB 2.1 MB/s eta 0:00:00
  Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
     -------------------------------------- 514.8/514.8 kB 2.0 MB/s eta 0:00:00
  Using cached P

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cpu requires torch==2.0.0, but you have torch 2.7.1 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
# import dependent libraries

import os
import torch
import whisper
import pandas as pd
import soundfile as sf
from datasets import Dataset, DatasetDict
from jiwer import wer, cer
from transformers import WhisperProcessor, WhisperForConditionalGeneration, TrainingArguments, Trainer
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torchaudio
import glob

c:\Users\ifedi\Documents\Applied AI and ML\Session 2\Playground\venv\my_pytorch_cpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ifedi\Documents\Applied AI and ML\Session 2\Playground\venv\my_pytorch_cpu\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
# Loading the data
import os
n = 0
for dirname, _, filenames in os.walk('../data-collection/'):
    for filename in filenames:
        if n < 3:
            print(os.path.join(dirname, filename))
            n += 1
        else: break
    if n >= 3: break

/kaggle/input/unitedsynmed/UnitedSynMed/transcript/validation.csv
/kaggle/input/unitedsynmed/UnitedSynMed/transcript/train.csv
/kaggle/input/unitedsynmed/UnitedSynMed/transcript/test.csv


In [20]:
# Paths to the dataset
audio_root = "/kaggle/input/unitedsynmed/UnitedSynMed/audio"
transcript_root = "/kaggle/input/unitedsynmed/UnitedSynMed/transcript/"

# # Load CSVs and match them with audio paths
# def load_split(split):
#     csv_path = os.path.join(transcript_root, f"{split}.csv")
#     df = pd.read_csv(csv_path)
#     df["path"] = df["file_name"].apply(lambda x: os.path.join(audio_root, split, x))
#     return df

# # Create datasets
# train_df = load_split("train")
# test_df = load_split("test")
# val_df = load_split("validation")

# # Convert to Hugging Face Dataset
# dataset = DatasetDict({
#     "train": Dataset.from_pandas(train_df),
#     "test": Dataset.from_pandas(test_df),
#     "validation": Dataset.from_pandas(val_df)
# })


In [21]:
dataset["train"][:5]

{'file_name': ['drug-female-defa7fcb-89d7-4b25-8834-90888b201d25.mp3',
  'drug-female-160727b4-dd0c-43c7-ba17-963ae54347a0.mp3',
  'drug-female-637d7dcc-fe73-499c-af76-b2ee28d36374.mp3',
  'drug-male-02a2daf6-0f99-4939-848d-adc95f03d4bd.mp3',
  'drug-brand-en-us-male-421229aa-4f71-48fa-bd43-a9ac606783f8.mp3'],
 'transcription': ['Durysta is a medication used to reduce eye pressure in patients with open-angle glaucoma or ocular hypertension.',
  'Annona muricata extract is known for its potential health benefits as a natural dietary supplement.',
  'Many patients have found relief with REDBURY GOLD for their ongoing health issues.',
  'ALMAL-Z is a popular medication used for treating allergies and cold symptoms.',
  ' Norfazole may cause side effects such as nausea or a metallic taste in the mouth.'],
 'path': ['/kaggle/input/unitedsynmed/UnitedSynMed/audio/train/drug-female-defa7fcb-89d7-4b25-8834-90888b201d25.mp3',
  '/kaggle/input/unitedsynmed/UnitedSynMed/audio/train/drug-female-16

In [10]:
from glob import glob

# Define source and target folders
source_root = "/kaggle/input/unitedsynmed/UnitedSynMed/audio"
target_root = "/kaggle/input/unitedsynmed/UnitedSynMed/audio_resampled"
target_sample_rate = 16000

# splits = ['train', 'test', 'validation']

# for split in splits:
#     src_dir = os.path.join(source_root, split)
#     tgt_dir = os.path.join(target_root, split)
#     os.makedirs(tgt_dir, exist_ok=True)

#     audio_files = glob.glob(os.path.join(src_dir, "*.mp3"))

    audio_files = glob(os.path.join(src_dir, "*.mp3"))

#         filename = os.path.splitext(os.path.basename(file))[0] + ".wav"
#         torchaudio.save(os.path.join(tgt_dir, filename), waveform, target_sample_rate)

# print("✅ All audio resampled and saved to:", target_root)

✅ All audio resampled and saved to: C:/Users/ifedi/Documents/Applied AI and ML/Session 2/INFO8665 - Projects in Machine Learning/SageCare-2.0/data-collection/UnitedSynMed/audio_resampled


In [ ]:

# Load Whisper processor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")

# Set target sample rate
target_sample_rate = 16000

def preprocess(batch):
    audio_input, sr = sf.read(batch["path"])
    
    # If the sample rate is not 16kHz, resample it
    waveform = torch.tensor(audio_input).float()
    if len(waveform.shape) > 1 and waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0)  # Convert to mono
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sample_rate)
    audio_input = resampler(waveform).numpy()
    
    inputs = processor(audio_input, sampling_rate=target_sample_rate, return_tensors="pt")
    batch["input_features"] = inputs.input_features[0]
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch

# Apply preprocessing
dataset = dataset.map(preprocess)

Map:   0%|          | 0/632548 [00:00<?, ? examples/s]

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100)
        batch["labels"] = labels

        return batch

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
# Freeze encoder layers
for param in model.model.encoder.parameters():
    param.requires_grad = False

In [ ]:

training_args = TrainingArguments(
    output_dir="./whisper-medical",
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    learning_rate=1e-4,
    warmup_steps=500,
    fp16=True,
    push_to_hub=False,
)

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer_score = wer(label_str, pred_str)
    cer_score = cer(label_str, pred_str)

    return {"wer": wer_score, "cer": cer_score}

results = trainer.evaluate()
print(results)

In [ ]:
model.save_pretrained("whisper-medical-finetuned")
processor.save_pretrained("whisper-medical-finetuned")